In [ ]:
import os

import pandas as pd

In [ ]:
models = [
    'CNN_EventCNN', 'CNN_Baseline', 'CNN_Light',
    'ParT_Baseline', 'ParT_Medium', 'ParT_Light', 'ParT_SuperLight', 'ParT_ExtremeLight'
]

print('- Model: $(\\eta, N_\\text{step}, N_\\text{accum}) \\Rightarrow (ACC_\\text{test}, AUC_\\text{test}, E_\\text{early})$')

for model in models:
    for date, suffix, channel, result_dir in [
        # ('20250713-173545', '', 'diphoton', 'ex-decay_jet-flavor'),
        # ('20250713-173106', '', 'diphoton', 'original_jet-flavor'),

        ('20250713-235925', 'in_decay', 'zz4l', 'original_jet-flavor'),
        ('20250713-230707', 'ex_decay', 'zz4l', 'ex-decay_jet-flavor'),
    ]:
        # Select the checkpoint directory based on the date and model.
        training_logs_dir = os.path.join(f"result-{channel}", result_dir, model)
        versions = os.listdir(training_logs_dir)
        try:
            ckpt_dir = [v for v in versions if date in v][0]
        except IndexError:
            continue

        # Hyperparameters of the training.
        _, lr, batch_size, seed = ckpt_dir.split('_')

        # Read the metrics CSV file of the training result.
        df = pd.read_csv(os.path.join(training_logs_dir, ckpt_dir, 'metrics.csv'))

        # The last row contains the test metrics.
        test_metrics = df.tail(1)
        test_accuracy = test_metrics['test_accuracy'].item()
        test_auc = test_metrics['test_auc'].item()
        early_stopped_epochs = test_metrics['epoch'].item()

        print(f"  - {model}: ({lr}, {batch_size}) $\\Rightarrow$ ({test_accuracy:.3f}, {test_auc:.3f}, {early_stopped_epochs}) # ({date}) {suffix}")